# Leash Bio - Predict New Medicines with BELKA

After 4 weeks, I'm still stuck with the CV strategy.
I created this Notebook to test different CV strategies.
1. Sample data. All positve cases plus 8 millions rows for no positives(9,5M rows * 3 Targets).
2. Simple Xgb model, no tunning and no GPU.
3. Different model for each target..
4. Split in two different problems, shared data (tests smiles exist in train) and nonshared data (tests smiles no exist in train).
  1. For shared data, suffle rows and split. 9M rows for trainig 500K for validation.
  2. For nonshare data,four folds of unique smiles, non shared bettein train and validation and rows with at least one positive case.
  
  
Version:
* V.8 - VarianceThreshold. select columns with Variance Threshold  0.01 ->training with 940 columns.
* V.9 - New down Sample data set, 4,5M rows(created like previous one of 9,5M). XGB model training iterations from 3000 to 1000. Variance Threshold 947 rows.
* V.11 - Back to 9,5M rows and 3000 iterations for training.



In [3]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import pickle
import random, os, gc
from scipy import sparse
from sklearn.metrics import average_precision_score

from xgboost import DMatrix
import xgboost as xgb


# Sample data based on https://www.kaggle.com/datasets/shlomoron/belka-shrunken-train-set

In [4]:
def seed_everything(seed: int):
    random.seed(seed)
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed) 
    
seed_everything(42)

In [5]:
def fold_properties(df):
    print('Shape :',df.shape)
    print('block1 :',df.buildingblock1_smiles.nunique())
    print('block2 :',df.buildingblock2_smiles.nunique())
    print('block3 :',df.buildingblock3_smiles.nunique())    
    for target in TARGETS:
        print(f'Positive Cases Ratio for {target} :', target,(df[target].sum()/df.shape[0]))

In [6]:
TARGETS=['binds_BRD4','binds_HSA','binds_sEH']
dtypes = {'buildingblock1_smiles': np.int16, 'buildingblock2_smiles': np.int16, 'buildingblock3_smiles': np.int16,
          'binds_BRD4':np.byte, 'binds_HSA':np.byte, 'binds_sEH':np.byte}

train = pd.read_csv('/kaggle/input/leashbio-10m-data-sample/train_sample.csv', dtype = dtypes, usecols=[0,1,2,4,5,6] )
train_index=train.index.tolist()
random.shuffle(train_index)
print('Train Sample Properties')
fold_properties(train)
    
#Load train_ecfp sparce matrix
train_ecfp = sparse.load_npz("/kaggle/input/leashbio-10m-data-sample/train_ecfp.npz")
print('train_ecfp Shape :',train_ecfp.shape)

train_B1_unique=train.buildingblock1_smiles.unique().tolist()
train_B2_unique=train.buildingblock2_smiles.unique().tolist()
train_B3_unique=train.buildingblock3_smiles.unique().tolist()
train_unique_smiles=set(train_B1_unique+train_B2_unique+train_B3_unique)


Train Sample Properties
Shape : (9509779, 6)
block1 : 271
block2 : 693
block3 : 872
Positive Cases Ratio for binds_BRD4 : binds_BRD4 0.048052010462072775
Positive Cases Ratio for binds_HSA : binds_HSA 0.04294631873148682
Positive Cases Ratio for binds_sEH : binds_sEH 0.07618810069087831
train_ecfp Shape : (9509779, 2048)


In [7]:
from sklearn.feature_selection import VarianceThreshold
for i in range(6):
    threshold=0.1/10**i
    var_thresh = VarianceThreshold(threshold=threshold)
    var_thresh.fit(train_ecfp[:100000].A)
    var_thresh_index=var_thresh.get_support()
    print(threshold,sum(var_thresh_index))


# select columns with Variance Threshold  001
    
#Apply mask to ecfp sparse matrix.
var_thresh = VarianceThreshold(threshold=0.005)
var_thresh.fit(train_ecfp[:100000].A)
var_thresh_index_1=var_thresh.get_support()
train_ecfp_1=train_ecfp[:,var_thresh_index_1]
print('train_ecfp Shape :',train_ecfp_1.shape)

0.1 97
0.01 967
0.001 1850
0.0001 1850
1e-05 1850
1e-06 1850
train_ecfp Shape : (9509779, 1373)


# Shared smiles Models

In [ ]:
iterations = 3000
early_stopping_rounds = 100
verbose_eval = 100

xgb_models=[]
valid_preds_score=[]
    
print('#*****#')
print(f'train Data')
train_fold_index=train_index[:9*10**6]
fold_properties(train.loc[train_fold_index])
X_tr=train_ecfp_1[(train_fold_index),:]


print('#*****#')
print('Valid Data')
valid_fold_index=train_index[9*10**6:]
fold_properties(train.loc[valid_fold_index])
X_val=train_ecfp_1[(valid_fold_index),:]
    
for i,target in enumerate(TARGETS):
    print(f'training target {target}')
    y_tr=train.loc[train_fold_index][target].values
    y_val=train.loc[valid_fold_index][target].values 
        
    xtrain = DMatrix(data=X_tr, label=y_tr)
    xvalid = DMatrix(data=X_val, label=y_val)
        
    scale_pos_weight=(len(y_tr)-y_tr.sum())/y_tr.sum()
    print('scale_pos_weight :',scale_pos_weight)
    
    xgb_params= { 'objective':'binary:logistic', 'eval_metric':'aucpr',
                 'learning_rate': 0.2, "n_jobs":12,"seed": 42,'colsample_bytree':0.8, 'scale_pos_weight':scale_pos_weight}

    XModel = xgb.train(xgb_params, xtrain,
                           evals=[(xvalid,'validation')],
                           verbose_eval=verbose_eval,
                           early_stopping_rounds=early_stopping_rounds,
                           xgb_model=None,
                           num_boost_round=iterations)
        

    y_pred_proba = XModel.predict(xvalid)  # Probability of the positive class
    map_score = average_precision_score(y_val, y_pred_proba)
    valid_preds_score.append(map_score)
    print(f"Mean Average Precision for Valid {target}: {map_score:.2f}")
    
    xgb_models.append(XModel)

#*****#
train Data
Shape : (9000000, 6)
block1 : 271
block2 : 693
block3 : 872
Positive Cases Ratio for binds_BRD4 : binds_BRD4 0.048081
Positive Cases Ratio for binds_HSA : binds_HSA 0.04292544444444445
Positive Cases Ratio for binds_sEH : binds_sEH 0.07618488888888889
#*****#
Valid Data
Shape : (509779, 6)
block1 : 271
block2 : 693
block3 : 871
Positive Cases Ratio for binds_BRD4 : binds_BRD4 0.047540208600197344
Positive Cases Ratio for binds_HSA : binds_HSA 0.043314848198925414
Positive Cases Ratio for binds_sEH : binds_sEH 0.07624480412100146
training target binds_BRD4
scale_pos_weight : 19.79823630956095
[0]	validation-aucpr:0.52363
[100]	validation-aucpr:0.78549
[200]	validation-aucpr:0.81716
[300]	validation-aucpr:0.83018
[400]	validation-aucpr:0.83773
[500]	validation-aucpr:0.84268
[600]	validation-aucpr:0.84556
[700]	validation-aucpr:0.84832
[800]	validation-aucpr:0.85031


In [ ]:
print('binds_BRD4 :',np.mean(valid_preds_score[0::3]))
print('binds_HSA :' ,np.mean(valid_preds_score[1::3]))
print('binds_sEH :' ,np.mean(valid_preds_score[2::3]))

In [ ]:
del train,xtrain,X_tr,xvalid,X_val,y_tr,y_val,valid_preds_score
gc.collect()

In [ ]:
preds_cols=['BRD4','HSA','sEH']
test=pd.read_parquet('/kaggle/input/leash-BELKA/test.parquet', engine = 'pyarrow')
blocks_dict=np.load('/kaggle/input/leashbio-10m-data-sample/blocks_dict.npy', allow_pickle=True)
blocks_dict = blocks_dict.tolist()
test['buildingblock1_smiles']=test['buildingblock1_smiles'].map(blocks_dict).values.astype('uint16')
test['buildingblock2_smiles']=test['buildingblock2_smiles'].map(blocks_dict).values.astype('uint16')
test['buildingblock3_smiles']=test['buildingblock3_smiles'].map(blocks_dict).values.astype('uint16')

test_preds=pd.DataFrame(test['molecule_smiles'].unique(),columns=['molecule_smiles'])
test_preds=test[['molecule_smiles','buildingblock1_smiles','buildingblock2_smiles','buildingblock3_smiles']].copy()
test_preds.drop_duplicates(inplace=True)
test_preds=test_preds.reset_index(drop=True) 

test_ecfp=sparse.load_npz("/kaggle/input/leashbio-10m-data-sample/test_ecfp.npz")
test_ecfp_1=test_ecfp[:,var_thresh_index_1]
print('test_ecfp Shape :',train_ecfp_1.shape)


test_B1_unique=test_preds.buildingblock1_smiles.unique().tolist()
test_B2_unique=test_preds.buildingblock2_smiles.unique().tolist()
test_B3_unique=test_preds.buildingblock3_smiles.unique().tolist()
test_unique_smiles=set(test_B1_unique+test_B2_unique+test_B3_unique)

test_preds[preds_cols]=np.zeros((len(test_preds),3))

print('Shape Test :', test.shape)
test_no_overlap=[bb for bb in test_unique_smiles if bb not in train_unique_smiles]
train_no_overlap=[bb for bb in train_unique_smiles if bb not in test_unique_smiles]
print('Test Block Unique :', len(test_unique_smiles))
print('Train Block Unique :', len(train_unique_smiles))
print('Test Block not in train :', len(test_no_overlap))
print('Train Block not in test :', len(train_no_overlap))

In [ ]:
for i,target in enumerate(TARGETS):    
    test_target=target.split('_')[1]      
    test_preds[test_target]=xgb_models[i].predict(DMatrix(data=test_ecfp_1))

test_BRD4=test_preds[['molecule_smiles','BRD4']].copy()
test_BRD4['protein_name']='BRD4'
test_BRD4=test_BRD4.rename(columns={"BRD4": "binds_1"})

test_HSA=test_preds[['molecule_smiles','HSA']].copy()
test_HSA['protein_name']='HSA'
test_HSA=test_HSA.rename(columns={"HSA": "binds_1"})

test_sEH=test_preds[['molecule_smiles','sEH']].copy()
test_sEH['protein_name']='sEH'
test_sEH=test_sEH.rename(columns={"sEH": "binds_1"})

test_preds_1=pd.concat([test_BRD4,test_HSA,test_sEH])
test=pd.merge(test,test_preds_1, on=['molecule_smiles','protein_name'],how = 'left')

In [ ]:
sample_submission = pd.read_csv("/kaggle/input/leash-BELKA/sample_submission.csv")
sample_submission['binds']=test['binds_1']
sample_submission.to_csv('submission.csv', index=False)
sample_submission.head()

In [ ]:
print(sample_submission.binds.max())
print(sample_submission.binds.min())
print(sample_submission.binds.mean())